In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle as pkl
import random
import plotly.express as px
import time
import seaborn as sns

In [2]:
pd.set_option('max_columns', None)

In [3]:
infile = open('..//3_k-means_clustering//apartment_data_frame_clustered_500_2021q2','rb')
frame = pkl.load(infile)
infile.close()

In [4]:
frame

,city,district,voivodeship,localization_y,localization_x,market,area,rooms,floor,floors,build_yr,price,url,price_of_sqm,cluster
0,Kraków,Podgórze,małopolskie,50.036500,20.015500,wtorny,46.30,2,6,8,2021.0,464000.0,https://gratka.pl/nieruchomosci/mieszkanie-kra...,10021.598272,242
1,Warszawa,Praga-Południe,mazowieckie,52.238088,21.115236,wtorny,80.91,2,2,5,2013.0,849000.0,https://gratka.pl/nieruchomosci/mieszkanie-war...,10493.140527,142
2,Katowice,Os. Paderewskiego,śląskie,50.252783,19.038932,pierwotny,44.21,2,1,9,2023.0,373100.0,https://gratka.pl/nieruchomosci/nowe-mieszkani...,8439.267134,326
3,Warszawa,Wola,mazowieckie,52.234980,20.961730,wtorny,34.00,1,1,5,1952.0,409900.0,https://gratka.pl/nieruchomosci/mieszkanie-war...,12055.882353,412
4,Poznań,Stare Miasto,wielkopolskie,52.453445,16.915134,wtorny,46.70,2,1,15,1984.0,329000.0,https://gratka.pl/nieruchomosci/mieszkanie-poz...,7044.967880,411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50411,Kołobrzeg,kołobrzeski,zachodniopomorskie,54.175917,15.583267,wtorny,27.00,2,parter,2,1948.0,340000.0,https://gratka.pl/nieruchomosci/mieszkanie-kol...,12592.592593,274
50412,Pruszków,pruszkowski,mazowieckie,52.155775,20.804646,wtorny,48.42,2,2,3,2019.0,539000.0,https://gratka.pl/nieruchomosci/mieszkanie-pru...,11131.763734,245
50413,Kraków,Os. Prądnik Biały,małopolskie,50.092900,19.926700,pierwotny,54.00,3,4,5,2020.0,499002.0,https://gratka.pl/nieruchomosci/nowe-mieszkani...,9240.777778,264
50414,Warszawa,Bielany,mazowieckie,52.267688,20.947620,wtorny,82.00,2,11,11,2001.0,940000.0,https://gratka.pl/nieruchomosci/mieszkanie-war...,11463.414634,350


## data preparation for ann

In [5]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50416 entries, 0 to 50415
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   city            50416 non-null  object 
 1   district        42465 non-null  object 
 2   voivodeship     50416 non-null  object 
 3   localization_y  50416 non-null  float64
 4   localization_x  50416 non-null  float64
 5   market          50416 non-null  object 
 6   area            50416 non-null  float64
 7   rooms           50416 non-null  object 
 8   floor           50416 non-null  object 
 9   floors          50416 non-null  object 
 10  build_yr        50416 non-null  float64
 11  price           50416 non-null  float64
 12  url             50416 non-null  object 
 13  price_of_sqm    50416 non-null  float64
 14  cluster         50416 non-null  int32  
dtypes: float64(6), int32(1), object(8)
memory usage: 5.6+ MB


In [6]:
frame = frame.drop(['city','district','voivodeship','localization_y','localization_x','price_of_sqm','url'],axis=1)

In [7]:
frame.cluster = frame.cluster.apply(str)

In [8]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50416 entries, 0 to 50415
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   market    50416 non-null  object 
 1   area      50416 non-null  float64
 2   rooms     50416 non-null  object 
 3   floor     50416 non-null  object 
 4   floors    50416 non-null  object 
 5   build_yr  50416 non-null  float64
 6   price     50416 non-null  float64
 7   cluster   50416 non-null  object 
dtypes: float64(3), object(5)
memory usage: 3.1+ MB


In [9]:
frame.rooms.value_counts().sort_index()

1                4427
2               16772
3               19442
4                7643
5                1620
6                 335
7                  91
8                   2
więcej niż 8       84
Name: rooms, dtype: int64

In [10]:
frame.floor.value_counts().sort_index()

1             11472
10              503
11              232
12              142
13               70
14               61
15               64
16               48
17               21
18               17
19               11
2              8886
20               23
21                7
22                9
23                7
24                7
25                6
26                3
27                6
28                5
29               23
3              7707
30                1
4              5668
5              2292
6              1497
7               980
8               818
9               572
parter         9240
powyżej 30       18
Name: floor, dtype: int64

In [11]:
frame.floors.value_counts().sort_index()

0 (parter)      117
1              2314
10             3300
11             1076
12              520
13              182
14              176
15              316
16              171
17              221
18               29
19                7
2              5682
20               15
21                3
22               61
23                8
24                6
25               14
26               62
27               43
28               15
29               60
3              8915
30                2
4             13714
5              5031
6              3406
7              2103
8              1919
9               869
powyżej 30       59
Name: floors, dtype: int64

In [12]:
frame.cluster.value_counts().sort_index()

0       34
1      495
10      84
100      5
101     22
      ... 
95      37
96      25
97      12
98      13
99      13
Name: cluster, Length: 500, dtype: int64

## creating dummy variables

In [13]:
dummies = pd.get_dummies(frame[['market','rooms','floor','floors','cluster']],drop_first=True)
frame = frame.drop(['market','rooms','floor','floors','cluster'],axis=1)
frame = pd.concat([frame,dummies],axis=1)

In [14]:
frame

,area,build_yr,price,market_wtorny,rooms_2,rooms_3,rooms_4,rooms_5,rooms_6,rooms_7,rooms_8,rooms_więcej niż 8,floor_10,floor_11,floor_12,floor_13,floor_14,floor_15,floor_16,floor_17,floor_18,floor_19,floor_2,floor_20,floor_21,floor_22,floor_23,floor_24,floor_25,floor_26,floor_27,floor_28,floor_29,floor_3,floor_30,floor_4,floor_5,floor_6,floor_7,floor_8,floor_9,floor_parter,floor_powyżej 30,floors_1,floors_10,floors_11,floors_12,floors_13,floors_14,floors_15,floors_16,floors_17,floors_18,floors_19,floors_2,floors_20,floors_21,floors_22,floors_23,floors_24,floors_25,floors_26,floors_27,floors_28,floors_29,floors_3,floors_30,floors_4,floors_5,floors_6,floors_7,floors_8,floors_9,floors_powyżej 30,cluster_1,cluster_10,cluster_100,cluster_101,cluster_102,cluster_103,cluster_104,cluster_105,cluster_106,cluster_107,cluster_108,cluster_109,cluster_11,cluster_110,cluster_111,cluster_112,cluster_113,cluster_114,cluster_115,cluster_116,cluster_117,cluster_118,cluster_119,cluster_12,cluster_120,cluster_121,cluster_122,cluster_123,cluster_124,cluster_125,cluster_126,cluster_127,cluster_128,cluster_129,cluster_13,cluster_130,cluster_131,cluster_132,cluster_133,cluster_134,cluster_135,cluster_136,cluster_137,cluster_138,cluster_139,cluster_14,cluster_140,cluster_141,cluster_142,cluster_143,cluster_144,cluster_145,cluster_146,cluster_147,cluster_148,cluster_149,cluster_15,cluster_150,cluster_151,cluster_152,cluster_153,cluster_154,cluster_155,cluster_156,cluster_157,cluster_158,cluster_159,cluster_16,cluster_160,cluster_161,cluster_162,cluster_163,cluster_164,cluster_165,cluster_166,cluster_167,cluster_168,cluster_169,cluster_17,cluster_170,cluster_171,cluster_172,cluster_173,cluster_174,cluster_175,cluster_176,cluster_177,cluster_178,cluster_179,cluster_18,cluster_180,cluster_181,cluster_182,cluster_183,cluster_184,cluster_185,cluster_186,cluster_187,cluster_188,cluster_189,cluster_19,cluster_190,cluster_191,cluster_192,cluster_193,cluster_194,cluster_195,cluster_196,cluster_197,cluster_198,cluster_199,cluster_2,cluster_20,cluster_200,cluster_201,cluster_202,cluster_203,cluster_204,cluster_205,cluster_206,cluster_207,cluster_208,cluster_209,cluster_21,cluster_210,cluster_211,cluster_212,cluster_213,cluster_214,cluster_215,cluster_216,cluster_217,cluster_218,cluster_219,cluster_22,cluster_220,cluster_221,cluster_222,cluster_223,cluster_224,cluster_225,cluster_226,cluster_227,cluster_228,cluster_229,cluster_23,cluster_230,cluster_231,cluster_232,cluster_233,cluster_234,cluster_235,cluster_236,cluster_237,cluster_238,cluster_239,cluster_24,cluster_240,cluster_241,cluster_242,cluster_243,cluster_244,cluster_245,cluster_246,cluster_247,cluster_248,cluster_249,cluster_25,cluster_250,cluster_251,cluster_252,cluster_253,cluster_254,cluster_255,cluster_256,cluster_257,cluster_258,cluster_259,cluster_26,cluster_260,cluster_261,cluster_262,cluster_263,cluster_264,cluster_265,cluster_266,cluster_267,cluster_268,cluster_269,cluster_27,cluster_270,cluster_271,cluster_272,cluster_273,cluster_274,cluster_275,cluster_276,cluster_277,cluster_278,cluster_279,cluster_28,cluster_280,cluster_281,cluster_282,cluster_283,cluster_284,cluster_285,cluster_286,cluster_287,cluster_288,cluster_289,cluster_29,cluster_290,cluster_291,cluster_292,cluster_293,cluster_294,cluster_295,cluster_296,cluster_297,cluster_298,cluster_299,cluster_3,cluster_30,cluster_300,cluster_301,cluster_302,cluster_303,cluster_304,cluster_305,cluster_306,cluster_307,cluster_308,cluster_309,cluster_31,cluster_310,cluster_311,cluster_312,cluster_313,cluster_314,cluster_315,cluster_316,cluster_317,cluster_318,cluster_319,cluster_32,cluster_320,cluster_321,cluster_322,cluster_323,cluster_324,cluster_325,cluster_326,cluster_327,cluster_328,cluster_329,cluster_33,cluster_330,cluster_331,cluster_332,cluster_333,cluster_334,cluster_335,cluster_336,cluster_337,cluster_338,cluster_339,cluster_34,cluster_340,cluster_341,cluster_342,cluster_343,cluster_344,cluster_345,cluster_346,cluster_347,cluster_348,clust

In [15]:
dummy_apartment_frame = frame.drop(index=frame.index[1:],axis=0)

In [16]:
type(dummy_apartment_frame)

pandas.core.frame.DataFrame

In [17]:
dummy_apartment_frame = dummy_apartment_frame.drop(['price'],axis=1)

In [18]:
dummy_apartment_frame

,area,build_yr,market_wtorny,rooms_2,rooms_3,rooms_4,rooms_5,rooms_6,rooms_7,rooms_8,rooms_więcej niż 8,floor_10,floor_11,floor_12,floor_13,floor_14,floor_15,floor_16,floor_17,floor_18,floor_19,floor_2,floor_20,floor_21,floor_22,floor_23,floor_24,floor_25,floor_26,floor_27,floor_28,floor_29,floor_3,floor_30,floor_4,floor_5,floor_6,floor_7,floor_8,floor_9,floor_parter,floor_powyżej 30,floors_1,floors_10,floors_11,floors_12,floors_13,floors_14,floors_15,floors_16,floors_17,floors_18,floors_19,floors_2,floors_20,floors_21,floors_22,floors_23,floors_24,floors_25,floors_26,floors_27,floors_28,floors_29,floors_3,floors_30,floors_4,floors_5,floors_6,floors_7,floors_8,floors_9,floors_powyżej 30,cluster_1,cluster_10,cluster_100,cluster_101,cluster_102,cluster_103,cluster_104,cluster_105,cluster_106,cluster_107,cluster_108,cluster_109,cluster_11,cluster_110,cluster_111,cluster_112,cluster_113,cluster_114,cluster_115,cluster_116,cluster_117,cluster_118,cluster_119,cluster_12,cluster_120,cluster_121,cluster_122,cluster_123,cluster_124,cluster_125,cluster_126,cluster_127,cluster_128,cluster_129,cluster_13,cluster_130,cluster_131,cluster_132,cluster_133,cluster_134,cluster_135,cluster_136,cluster_137,cluster_138,cluster_139,cluster_14,cluster_140,cluster_141,cluster_142,cluster_143,cluster_144,cluster_145,cluster_146,cluster_147,cluster_148,cluster_149,cluster_15,cluster_150,cluster_151,cluster_152,cluster_153,cluster_154,cluster_155,cluster_156,cluster_157,cluster_158,cluster_159,cluster_16,cluster_160,cluster_161,cluster_162,cluster_163,cluster_164,cluster_165,cluster_166,cluster_167,cluster_168,cluster_169,cluster_17,cluster_170,cluster_171,cluster_172,cluster_173,cluster_174,cluster_175,cluster_176,cluster_177,cluster_178,cluster_179,cluster_18,cluster_180,cluster_181,cluster_182,cluster_183,cluster_184,cluster_185,cluster_186,cluster_187,cluster_188,cluster_189,cluster_19,cluster_190,cluster_191,cluster_192,cluster_193,cluster_194,cluster_195,cluster_196,cluster_197,cluster_198,cluster_199,cluster_2,cluster_20,cluster_200,cluster_201,cluster_202,cluster_203,cluster_204,cluster_205,cluster_206,cluster_207,cluster_208,cluster_209,cluster_21,cluster_210,cluster_211,cluster_212,cluster_213,cluster_214,cluster_215,cluster_216,cluster_217,cluster_218,cluster_219,cluster_22,cluster_220,cluster_221,cluster_222,cluster_223,cluster_224,cluster_225,cluster_226,cluster_227,cluster_228,cluster_229,cluster_23,cluster_230,cluster_231,cluster_232,cluster_233,cluster_234,cluster_235,cluster_236,cluster_237,cluster_238,cluster_239,cluster_24,cluster_240,cluster_241,cluster_242,cluster_243,cluster_244,cluster_245,cluster_246,cluster_247,cluster_248,cluster_249,cluster_25,cluster_250,cluster_251,cluster_252,cluster_253,cluster_254,cluster_255,cluster_256,cluster_257,cluster_258,cluster_259,cluster_26,cluster_260,cluster_261,cluster_262,cluster_263,cluster_264,cluster_265,cluster_266,cluster_267,cluster_268,cluster_269,cluster_27,cluster_270,cluster_271,cluster_272,cluster_273,cluster_274,cluster_275,cluster_276,cluster_277,cluster_278,cluster_279,cluster_28,cluster_280,cluster_281,cluster_282,cluster_283,cluster_284,cluster_285,cluster_286,cluster_287,cluster_288,cluster_289,cluster_29,cluster_290,cluster_291,cluster_292,cluster_293,cluster_294,cluster_295,cluster_296,cluster_297,cluster_298,cluster_299,cluster_3,cluster_30,cluster_300,cluster_301,cluster_302,cluster_303,cluster_304,cluster_305,cluster_306,cluster_307,cluster_308,cluster_309,cluster_31,cluster_310,cluster_311,cluster_312,cluster_313,cluster_314,cluster_315,cluster_316,cluster_317,cluster_318,cluster_319,cluster_32,cluster_320,cluster_321,cluster_322,cluster_323,cluster_324,cluster_325,cluster_326,cluster_327,cluster_328,cluster_329,cluster_33,cluster_330,cluster_331,cluster_332,cluster_333,cluster_334,cluster_335,cluster_336,cluster_337,cluster_338,cluster_339,cluster_34,cluster_340,cluster_341,cluster_342,cluster_343,cluster_344,cluster_345,cluster_346,cluster_347,cluster_348,cluster_349

In [19]:
dummy_apartment_frame.replace(dummy_apartment_frame.iloc[0],0,inplace=True)

In [20]:
list(dummy_apartment_frame.columns)

['area',
 'build_yr',
 'market_wtorny',
 'rooms_2',
 'rooms_3',
 'rooms_4',
 'rooms_5',
 'rooms_6',
 'rooms_7',
 'rooms_8',
 'rooms_więcej niż 8',
 'floor_10',
 'floor_11',
 'floor_12',
 'floor_13',
 'floor_14',
 'floor_15',
 'floor_16',
 'floor_17',
 'floor_18',
 'floor_19',
 'floor_2',
 'floor_20',
 'floor_21',
 'floor_22',
 'floor_23',
 'floor_24',
 'floor_25',
 'floor_26',
 'floor_27',
 'floor_28',
 'floor_29',
 'floor_3',
 'floor_30',
 'floor_4',
 'floor_5',
 'floor_6',
 'floor_7',
 'floor_8',
 'floor_9',
 'floor_parter',
 'floor_powyżej 30',
 'floors_1',
 'floors_10',
 'floors_11',
 'floors_12',
 'floors_13',
 'floors_14',
 'floors_15',
 'floors_16',
 'floors_17',
 'floors_18',
 'floors_19',
 'floors_2',
 'floors_20',
 'floors_21',
 'floors_22',
 'floors_23',
 'floors_24',
 'floors_25',
 'floors_26',
 'floors_27',
 'floors_28',
 'floors_29',
 'floors_3',
 'floors_30',
 'floors_4',
 'floors_5',
 'floors_6',
 'floors_7',
 'floors_8',
 'floors_9',
 'floors_powyżej 30',
 'cluster_1',

## saving the frame

In [21]:
outfile = open('dummy_apartment_frame_2021q2','wb')
pkl.dump(dummy_apartment_frame,outfile)
outfile.close()

## ann estimation

In [22]:
from tensorflow.keras.models import load_model
import joblib

In [23]:
model = load_model('models_ann//2021q2_500_2021-07-13--17-29')

In [24]:
scaler = joblib.load('scaler_500_2021q2')

In [25]:
dummy_apartment_frame.area = 50
dummy_apartment_frame.build_yr = 2009
dummy_apartment_frame.market_wtorny = 1
dummy_apartment_frame.rooms_2 = 1
dummy_apartment_frame.floor_2 = 1
dummy_apartment_frame.floors_4 = 1
dummy_apartment_frame.cluster_412 = 1

In [26]:
dummy_apartment_frame

,area,build_yr,market_wtorny,rooms_2,rooms_3,rooms_4,rooms_5,rooms_6,rooms_7,rooms_8,rooms_więcej niż 8,floor_10,floor_11,floor_12,floor_13,floor_14,floor_15,floor_16,floor_17,floor_18,floor_19,floor_2,floor_20,floor_21,floor_22,floor_23,floor_24,floor_25,floor_26,floor_27,floor_28,floor_29,floor_3,floor_30,floor_4,floor_5,floor_6,floor_7,floor_8,floor_9,floor_parter,floor_powyżej 30,floors_1,floors_10,floors_11,floors_12,floors_13,floors_14,floors_15,floors_16,floors_17,floors_18,floors_19,floors_2,floors_20,floors_21,floors_22,floors_23,floors_24,floors_25,floors_26,floors_27,floors_28,floors_29,floors_3,floors_30,floors_4,floors_5,floors_6,floors_7,floors_8,floors_9,floors_powyżej 30,cluster_1,cluster_10,cluster_100,cluster_101,cluster_102,cluster_103,cluster_104,cluster_105,cluster_106,cluster_107,cluster_108,cluster_109,cluster_11,cluster_110,cluster_111,cluster_112,cluster_113,cluster_114,cluster_115,cluster_116,cluster_117,cluster_118,cluster_119,cluster_12,cluster_120,cluster_121,cluster_122,cluster_123,cluster_124,cluster_125,cluster_126,cluster_127,cluster_128,cluster_129,cluster_13,cluster_130,cluster_131,cluster_132,cluster_133,cluster_134,cluster_135,cluster_136,cluster_137,cluster_138,cluster_139,cluster_14,cluster_140,cluster_141,cluster_142,cluster_143,cluster_144,cluster_145,cluster_146,cluster_147,cluster_148,cluster_149,cluster_15,cluster_150,cluster_151,cluster_152,cluster_153,cluster_154,cluster_155,cluster_156,cluster_157,cluster_158,cluster_159,cluster_16,cluster_160,cluster_161,cluster_162,cluster_163,cluster_164,cluster_165,cluster_166,cluster_167,cluster_168,cluster_169,cluster_17,cluster_170,cluster_171,cluster_172,cluster_173,cluster_174,cluster_175,cluster_176,cluster_177,cluster_178,cluster_179,cluster_18,cluster_180,cluster_181,cluster_182,cluster_183,cluster_184,cluster_185,cluster_186,cluster_187,cluster_188,cluster_189,cluster_19,cluster_190,cluster_191,cluster_192,cluster_193,cluster_194,cluster_195,cluster_196,cluster_197,cluster_198,cluster_199,cluster_2,cluster_20,cluster_200,cluster_201,cluster_202,cluster_203,cluster_204,cluster_205,cluster_206,cluster_207,cluster_208,cluster_209,cluster_21,cluster_210,cluster_211,cluster_212,cluster_213,cluster_214,cluster_215,cluster_216,cluster_217,cluster_218,cluster_219,cluster_22,cluster_220,cluster_221,cluster_222,cluster_223,cluster_224,cluster_225,cluster_226,cluster_227,cluster_228,cluster_229,cluster_23,cluster_230,cluster_231,cluster_232,cluster_233,cluster_234,cluster_235,cluster_236,cluster_237,cluster_238,cluster_239,cluster_24,cluster_240,cluster_241,cluster_242,cluster_243,cluster_244,cluster_245,cluster_246,cluster_247,cluster_248,cluster_249,cluster_25,cluster_250,cluster_251,cluster_252,cluster_253,cluster_254,cluster_255,cluster_256,cluster_257,cluster_258,cluster_259,cluster_26,cluster_260,cluster_261,cluster_262,cluster_263,cluster_264,cluster_265,cluster_266,cluster_267,cluster_268,cluster_269,cluster_27,cluster_270,cluster_271,cluster_272,cluster_273,cluster_274,cluster_275,cluster_276,cluster_277,cluster_278,cluster_279,cluster_28,cluster_280,cluster_281,cluster_282,cluster_283,cluster_284,cluster_285,cluster_286,cluster_287,cluster_288,cluster_289,cluster_29,cluster_290,cluster_291,cluster_292,cluster_293,cluster_294,cluster_295,cluster_296,cluster_297,cluster_298,cluster_299,cluster_3,cluster_30,cluster_300,cluster_301,cluster_302,cluster_303,cluster_304,cluster_305,cluster_306,cluster_307,cluster_308,cluster_309,cluster_31,cluster_310,cluster_311,cluster_312,cluster_313,cluster_314,cluster_315,cluster_316,cluster_317,cluster_318,cluster_319,cluster_32,cluster_320,cluster_321,cluster_322,cluster_323,cluster_324,cluster_325,cluster_326,cluster_327,cluster_328,cluster_329,cluster_33,cluster_330,cluster_331,cluster_332,cluster_333,cluster_334,cluster_335,cluster_336,cluster_337,cluster_338,cluster_339,cluster_34,cluster_340,cluster_341,cluster_342,cluster_343,cluster_344,cluster_345,cluster_346,cluster_347,cluster_348,cluster_349

In [27]:
input_data = scaler.transform(dummy_apartment_frame)

In [28]:
input_data

array([[0.09090909, 0.87903226, 1.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [29]:
model.predict(input_data)

array([[672980.25]], dtype=float32)

## random forest estimation

In [30]:
model_rf = joblib.load('..//4_svm&random_forest_estimators//random_forest_model_2021q2')

In [31]:
model_rf.predict(input_data)

array([251013.3])